#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   (quando tiver)


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [4]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao

        produtos.append(produto)
    return produtos


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns).text
            
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [9]:
read = pd.read_excel('upseller_template.xlsx')
produtos = read[-1:]


for  index, produto in produtos.iterrows():
    # fornecedor = produto['Fornecedor']
    # codigo_produto = produto['Codigo Produto']
    # descricao = produto['Descrição']
    try:
    #     if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
    #         url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
    #     elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
    #         url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
    #     elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
    #         url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
    #     else:
    #         print('Indisponivel')
    #         continue
        
        url = 'https://construja.com.br/produto/101223/estrela-tapa-furo-ll-20mm'

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}
       
        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        comprimento = extrai.get("comprimento")

        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        categoria = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "SUB CATEGORIA"), "Não disponível")
        print(categoria)
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)




CONEXÕES ÁGUA
name 'codigo_barras' is not defined


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

# === Configurações ===
threshold = 75  # similaridade mínima
arquivo_produtos = "produtos.xlsx"
arquivo_saida_bling = "produtos_para_bling.xlsx"

# === Leitura inicial ===
df = pd.read_excel(arquivo_produtos)

# === Agrupamento Fuzzy ===
grupos, grupo_id, grupo_coluna = {}, 1, []

for desc in df["Descrição"]:
    # Se já está em algum grupo, apenas reutiliza o ID
    gid_existente = next((gid for gid, itens in grupos.items() if desc in itens), None)
    if gid_existente:
        grupo_coluna.append(gid_existente)
        continue

    # Cria novo grupo
    grupos[grupo_id] = [desc]
    grupo_coluna.append(grupo_id)

    # Busca similares
    for match_desc, score, _ in process.extract(desc, df["Descrição"], scorer=fuzz.token_sort_ratio, limit=None):
        if score >= threshold and match_desc != desc:
            grupos[grupo_id].append(match_desc)

    grupo_id += 1

df["Grupo"] = grupo_coluna

# Filtra linhas com peso válido
df = df[df["peso"].astype(str).str.upper() != "NÃO DISPONIVEL"]

# Padroniza colunas
df.columns = df.columns.str.strip().str.lower()

# Garante nome da coluna de código
if "codigo" not in df.columns:
    for c in df.columns:
        if "código" in c or "codigo" in c:
            df.rename(columns={c: "codigo"}, inplace=True)
            break

# Cria estrutura de hierarquia
bling_data = [
    {"codigo": row["codigo produto"], "codigo pai": "" if idx == 0 else grupo.iloc[0]["codigo produto"]}
    for _, grupo in df.groupby("grupo") 
    for idx, row in grupo.reset_index(drop=True).iterrows()
]

bling_df = pd.DataFrame(bling_data)

# Junta tudo
bling_final = df.merge(bling_df, on="codigo", how="right")
bling_final.to_excel(arquivo_saida_bling, index=False)

print("✅ Planilha gerada com sucesso:", arquivo_saida_bling)


In [ ]:
import pandas as pd 

df = pd.read_excel('produtos.xlsx')

marcas = df['Marca'].unique().tolist()
marcas

marcas remove 

['KRONA',
 'NOVA TINTAS',
 'PIX',
 'OVERTIME',
 'HERC',
 'NEW-FIX',
 'BLUKIT',
 'COLORGIN',
 'CHAMEX',
 'LUKMA',
 'ESTRELA',
 'ASTRA',
 'CASCOLA',
 'VIQUA',
 'FERJA',
 'VEDACIT',
 'GERDAU',
 'CORTAG',
 'INJEREST',
 'LORENZETTI',
 nan,
 'PADO',
 'TRAMON',
 'XADREZ',
 'HENKEL',
 'ATLAS',
 'ALIANÇA',
 'QUALITRONIX',
 'ILUMI',
 'IRWIN',
 'TEK BOND',
 'SANRO',
 'ANCORA',
 'SPARTA',
 'IBIRA',
 'ESFERA',
 'PERFECT',
 'UNIPEGA',
 'GARIN',
 'SUPORTES IGUATEMI',
 'DETONI',
 'MAX',
 'EDA',
 'EUCALAR',
 'FC',
 'TAF',
 'VIAPOL',
 'EUCATEX',
 'QUARTZOLIT',
 'FAME',
 'TIGRE',
 'ADERE',
 'DRYKO',
 'EXCELLENT',
 'ALPHA',
 'MULTILIT',
 'ACQUAFLEX',
 'ULTRA LIGHT',
 'COTLIN',
 'MAXI RUBBER',
 'LINHAL',
 'AMANCO',
 'GRAP',
 'SILVANA',
 'ITAQUA',
 'MGM',
 'STARRETT',
 'ALUMASA',
 'ARGEVVISION',
 'BRASFORMA',
 'FAMASTIL',
 'FOXLUX',
 'THOMPSON',
 'NAUTIKA',
 'JUNTALIDER',
 'STECK',
 'KALIPSO',
 'FORTLEV',
 'NORTON',
 'JNG',
 'VEDATUDO',
 'TRIONITE',
 'RENDICOLLA',
 'MEGATRON',
 'ARUJA',
 'RAYCO',
 'OUROFIX'

In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from env import fornecedores_noweb_scapping

# === 1. Carregar o arquivo ===
df = pd.read_excel('produtos.xlsx')

# === 2. Função para limpar e normalizar texto ===
def limpar_texto(txt):
    if not isinstance(txt, str):

        return ""
    # Remove acentos
    txt = unicodedata.normalize('NFD', txt)
    txt = txt.encode('ascii', 'ignore').decode('utf-8')
    # Converte para maiúsculas
    txt = txt.upper()
    # Remove caracteres especiais
    txt = re.sub(r'[^A-Z0-9 ]', '', txt)
    # Remove espaços duplicados
    txt = re.sub(r'\s+', ' ', txt).strip()
    return txt


# === 3. Lista de marcas fixas (forçadas a fazer parte da análise) ===
marcas_adicionais = [
    'SHIVA','BRASCOLA','FERTAK','NATICON','NOBRE','ADELBRAS','STA MARINA','FIRMEZA','OLIPLAS','STAM', 'R.C.A','CORONA','TOP FIO',
    'MAKITA','REXON','SOPRANO','FAST LUB','ACQUA','KIAN','SOUDAL','DEPLAST','TATU','GRENDHA', 'PANASONIC','PAULICEIA','MORIA',
    'ALLTAPE','PLASTILIT','TITANIUM','FERRARI','NEGREIRA','RADIAL','BLUMENAU','J.F METAIS','ART METAIS','LONAX', 'GHEL PLUS',
    'CHEMICOLOR','SANY','PROAQUA','BICROM','WAGO','TECHNA','METASUL','GARAPEIRA','ROCO','PRO FOAM','DURACELL','POLIBEL','AGELUX',
    'REMOX','RORATO','WAVES','LEGRAND','PAPAIZ','AGELUZ','IMPERIAL','HYDRA','KITUBO','NETTE','SANLIMP', 'EMA METAIS','M&M'
    'BEARS','ADTEX','GALO','PEGAFORTE','EMAVE','ELGIN','LUCONI','VEGA','MUNDIAL','DELTA', 'COBRECOM','PLASTBIG','FERE','SATA',
    'ALUMBRA', 'FIOLUX','PACETTA','ITAMBE','PEXCEL','EMAVA','KADESH','IBERE','GOLD','PERLEX','IMA','SAFETY','PIAL','PLASTIK',
    'OUROLUX', 'LED BEE','BELZER','PROTEG','NASTRO','FJ FERR', 'SAO RAFAEL', 'STILLUS','FLASH LIMP', 'ZUMPLAST','REDY','PRIMETECH',
    'SECALUX', 'PLASTIC','SUPER BONDER', 'ARTPLAS','FABER CASTELL','GUARANI','DEWALT','ALUREM','PERFIX ','STELL','FERROX','USAF',
    'INJET','CMK','BEARS','MINASUL','DEGOMASTER','SIENA'
]
marcas_adicionais = [limpar_texto(m) for m in marcas_adicionais]


# === 4. Dicionário de sinônimos / variações ===
mapa_variacoes = {
    'TRAMONTINA': ['TRAMON', 'TRAMONTINA', 'TRAMONTINA ELE', 'TRAMONTINA GARI', 'TRAMONTINA MULT', 'TRAM'],
    'JOMARCA': ['JOMARCA', 'JOMARC', 'JOMARCA JMC', 'JMC'],
    'NEW FIX': ['NEW FIX', 'NEWFIX', 'NEW-FIX', 'NEW F','PARAF CHIPBOARD CHATA PHS','PARAF SEX.ROS.SOBERBA'],
    'NOVA': ['NOVA TINTAS', 'NOVA'],
    'DRYKO': ['DRYKO', 'DRYKOPRIMER'],
    'SIKA': ['SIKATOP', 'SIKA'],
    'BRASFORT': ['BRASF', 'BRASFORT'],
    'ILUMI': ['ILUM', 'ILUMI'],
    'DENVER': ['DENVER','DENVERIMPER', 'DENVERLAJE','DENVERTEC','DENVERCRILL', 'DENVERFITA','DENVERFIX'],
    'THOMPSOM': ['THOMPSOM', 'THOMP'],
    'PEGAFORTE': ['PEGAFORTE', 'PEGA FORTE'],
    'CALHA FORTE': ['CALHA FORTE', 'CALHA FORT'],
    'VEDACIT': ['VEDACIT', 'BIANCO'],
    'LORENZETTI': ['LOREN','LORENZETTI','ORIG.L&C D','.PMR D.','.L&C '],
    'KADESH': ['BOT FLEX ELASTICO PRETA BI']
}


# === 5. Montar lista final de marcas conhecidas ===
marcas_planilha = df['Marca'].dropna().unique().tolist()
marcas_planilha.remove('5+')
marcas_planilha = [limpar_texto(m) for m in marcas_planilha if isinstance(m, str) and m.strip()]

# Combinar todas
marcas = list(set(marcas_planilha + marcas_adicionais))

# Adicionar também as variações do mapa
for marca_principal, variacoes in mapa_variacoes.items():
    marcas.append(marca_principal)
    marcas.extend([limpar_texto(v) for v in variacoes])

marcas = list(set(marcas))


# === 7. Função para detectar marca na descrição ===
def detectar_marca(descricao):
    desc_limpa = limpar_texto(descricao)

    # Verifica variações conhecidas primeiro
    for marca_padrao, variacoes in mapa_variacoes.items():
        for v in variacoes:
            if limpar_texto(v) in desc_limpa:
                return marca_padrao

    # Caso não encontre nas variações, busca direta na lista geral
    for marca in marcas:
        if marca and marca in desc_limpa:
            return marca

    return np.nan

df_filtrado = df.copy()

# === 8. Detectar e preencher marcas ausentes ===
df_filtrado['Marca Detectada'] = df_filtrado['Descrição'].apply(detectar_marca)

# Preenche apenas onde está vazio
df_filtrado['Marca'] = df_filtrado['Marca'].fillna(df_filtrado['Marca Detectada'])

df_filtrado['Marca'] = df_filtrado['Marca'].fillna('GENÉRICO')
# Atualiza no dataframe principal
df.update(df_filtrado)


# === 9. Salvar resultado ===
df.to_excel('produtos_com_marcas.xlsx', index=False)

# === 10. Relatório simples ===
print("✅ Processamento concluído!")
print(f"Total de produtos analisados: {len(df_filtrado)}")
print(f"Marcas detectadas automaticamente: {df_filtrado['Marca Detectada'].notna().sum()}")

# Top 10 marcas mais detectadas
print("\nTop marcas detectadas:")
print(df_filtrado['Marca Detectada'].value_counts().head(10))


✅ Processamento concluído!
Total de produtos analisados: 4436
Marcas detectadas automaticamente: 4219

Top marcas detectadas:
Marca Detectada
NEW FIX       157
TRAMONTINA    139
EDA           124
FERJA         109
KRONA          88
SIL            87
LORENZETTI     85
BRASFORT       79
FORTLEV        72
PLASTILIT      70
Name: count, dtype: int64
